[source](https://arxiv.org/pdf/1310.4546.pdf)

# Highlights
* subsampling frequent words (2 - 10x speedup + improve representation of less frequent words)
    * Noise Contrastive Estimation (NCE), see [notes](http://demo.clab.cs.cmu.edu/cdyer/nce_notes.pdf)
* neg sampling
* simple method for finding phrases in text
* intuition for the **additive compositionality** of word vectors

## Skip-Gram Model

* The objective of skip-gram is to minimize $\frac{1}{T} \sum_{t=1}^T \sum_{-c <= j <= c, j \neq 0} log p(w_{t+j} | w_t) $

* $p(w_o | w_i)$ is defined via $$ \frac{ exp(v^T_{wo} v_{wi}) }{\sum_{w=1}^W exp(v^T_{w} v_{wi})} $$
* not computationally efficient, because it runs in $O(W)$ where roughly $10^5 <= W <= 10^7 $

### Hierarchical Softmax
* computational approxiamation of the full softmax
* needs to compute $O(log W)$ instead of $O(W)$
* uses a binary tree representation with $W$ leaves, each node represent the probabilities of the child nodes
* let $n(w,j)$ be the jth node from the root to the node w, $L(w)$ be the length of the path
    * note: $n(w, 1)$ is the root, and $n(w,L(w)) = w$
* $$ p(w | w_I) = \prod_{j=1}^{L(w)} \sigma([n(w, j+1) = ch(n(w,j))] v_{n(w,j)}^T v_{wI}  ) $$
* where $\sigma$ is the sigmoid and $[]$ is the identity and $ch()$ means child of
* use a binary **Huffman tree**, as it assigns short codes to the frequent words which results in fast training

## Negative Sampling

* Noise Contrastive Estimation (NCE)
    * a good model should be able to differentiate data from noise by means of logistic regression
* $$ log \sigma(v_{wO}^T v_{wI}) + \sum_i^k E_{w_i \thicksim P_n(w)} [log \sigma(-v_{wi}^T v_{wI} )] $$ 
* notice the **"-"** in the sum
* idea is to distinguish data from noise distribution (k negative samples) 
    * noise distribution $P_n(w)$ is sampled via logisitc regression
    * neg sampling just uses the unigram distribution to the 3/4 power

## Subsampling Frequent Words

* A lot of words appear too often but don't have any practical meaning e.g. the
* justifications for removing frequent words with probability
    * observing "France" and "the" just isn't very meaningful
    * stop words (or high freq ones) usually don't change in vector representation
* simple subsampling approach:
    * each word is disregarded with probability $P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}}$
    * t is a threshold and $f(w_i)$ is the frequency
    * formula chosen heurestically, but works well
* accelerates learning and improves acc of rare words

## Learning Phrases

* e.g. "NY Times" "Toronto Maple Leafs"
* $$ score(w_i, w_j) = \frac{count(w_i, w_j) - \delta }{count(w_i) * count(w_j)} $$
* $\delta$ is a discounting coefficient, prevents too many phrases consisting of very infrequent words to be formed
* merge words into bigrams with a simple threshold
* run 2-4 times to form $n > 2$ n-grams

## Additive Compositionality

* e.g. "America" + "capital" = "Washington"
* The additive property of the vectors can be explained by inspecting the training objective
    * softmax $p(w_o | w_i)$ is defined via $$ \frac{ exp(v^T_{wo} v_{wi}) }{\sum_{w=1}^W exp(v^T_{w} v_{wi})} $$
    * the dot product means that the word vectors are in a linear relationship
    * since we are minimizing $log p(w_o | w_i)$ over the dataset, the vectors are representing the distribution where the word appears
    * **????** These values are related logarithmically to the probabilities computed by the output layer, so the sum of two word vectors is related to the product of the two context distributions
    * The product works here as the AND function: words that are assigned high probabilities by both word vectors will have high probability, and the other words will have low probability. 
    * Thus, if “Volga River” appears frequently in the same sentence together with the words “Russian” and “river”, the sum of these two word vectors will result in such a feature vector that is close to the vector of “Volga River”

## Extra notes on the Additive Compositionality:

* (just a random thought) similarity between softmax and covariance $$ \frac{v v^T}{E(V V^T)} = \frac{v v^T}{\Sigma} $$
* softmax and bayes rule connection $$ \frac{P(x, y)}{P(y)} = \frac{P(x_j,y)}{\sum_i P(x_i, y)} = \frac{exp(x_j^T y)}{\sum_i exp(x_i^T y)} $$
    * however, note they are not totally the same
    * $P(x, y)$ is always in the confines of probability $0 <= P() <= 1$
    * there's the exponential (useful to deal with the log)
* we can think of $exp(v_j^T v_I)$ as $exp(\theta^T v_j)$
    * so since the contexts are combined in a log linear fashion, $\sum_{-c <= j <= j} log(P(w_j | w_I))$, if we ignore the denom of the softmax, it becomes $\sum_j \theta^T v_j$
    * so we can think of the skip-gram objective as $\text{arg max} \sum_j \theta^T v_j$ 
    * if $\theta$ is the input word, then maximizing the objective not only makes $v_j$ as close to $v_I$ as possible, but also $v_{j1}$ and $v_{j2}$ as close to each other as possible
* "Russian" + "river" = "Volga River" intuition:
    * in skip-gram, whenever we maximize $P(w_j | w_I)$, we will also maximize $P(w_I | w_j)$ because of the symmetric nature of the window function 
    * if A, B and C frequently appear together, then A + B should be closer to C than A or B individually
    * **(JASON DOUBLE CHECK THIS)** I think what this implies is that "A + B = C" occurs because "A B C" occurs often, not because w2v learns magically that "A C" and "B C" both appear, therefore "A + B = C"